# Beta Version
Create a mask to get only the face

In [9]:
import cv2
import sys
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image, ImageFilter, ImageDraw
import face_recognition
import dlib
from skimage import measure  
import scipy.ndimage as ndi  

In [15]:
# Load the jpg file into a numpy array
image = face_recognition.load_image_file("test.jpg")

face_locations = face_recognition.face_locations(image)
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_locations)))

I found 2 face(s) in this photograph.


In [21]:
for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}" \
          .format(top, left, bottom, right))

A face is located at pixel location Top: 199, Left: 724, Bottom: 307, Right: 832
A face is located at pixel location Top: 72, Left: 162, Bottom: 146, Right: 237


In [197]:
mask = np.zeros(image.shape, dtype=np.uint8)

In [199]:
# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

# Create a PIL imagedraw object so we can draw on the picture
pil_image = Image.fromarray(mask)#image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    left_eyebrow_top = min(face_landmarks['left_eyebrow'], key = lambda t: t[1])
    right_eyebrow_top = min(face_landmarks['right_eyebrow'], key = lambda t: t[1])
    
    points = []
    points.insert(0, left_eyebrow_top)
    points.insert(0, right_eyebrow_top)
    points.insert(0, face_landmarks['chin'])
    
    cleaned = [j for i in points for j in i]
    
    d.polygon(cleaned, fill = 'white')

# Show the picture
pil_image.show()
pil_image = pil_image.save("mask.jpg") 

I found 2 face(s) in this photograph.


In [201]:
def soft_blur_with_mask(image: np.ndarray, mask: np.ndarray) -> np.ndarray:
    assert len(mask.shape) == 2, mask.shape
    # Create a blurred copy of the original image. This can take up to 1-2 seconds today, 
    # because the image is big (~5-10 Megapixels)
    blurred_image = cv2.GaussianBlur(image, (221, 221), sigmaX=20, sigmaY=20)
    image_height, image_width = image.shape[:2]
    mask = cv2.resize(mask.astype(np.uint8), (image_width, image_height), 
                      interpolation=cv2.INTER_NEAREST)
    # Blurring the mask itself to get a softer mask with no firm edges
    mask = cv2.GaussianBlur(mask.astype(np.float32), (11, 11), 10, 10)[:, :, None]
    mask = mask/255.0

    # Take the blurred image where the mask it positive, and the original image where 
    # the image is original
    return (mask * blurred_image + (1.0 - mask) * image).clip(0, 255.0).astype(np.uint8)

In [ ]:
soft_blur_with_mask(image, )